In [1]:

import sys
sys.path.append('src/')
from src.logic_utils import get_lang

import torch
import numpy as np

In [2]:
lark_path = '../src/lark/exp.lark'
lang_base_path = '../data/lang/'
lang, clauses, bk, atoms = get_lang(
        lark_path, lang_base_path,'coinjump1', 'coinjump')

In [3]:
atoms

[.(__F__),
 .(__T__),
 closeby(obj1,obj2),
 closeby(obj1,obj3),
 closeby(obj1,obj4),
 closeby(obj2,obj1),
 closeby(obj2,obj3),
 closeby(obj2,obj4),
 closeby(obj3,obj1),
 closeby(obj3,obj2),
 closeby(obj3,obj4),
 closeby(obj4,obj1),
 closeby(obj4,obj2),
 closeby(obj4,obj3),
 have_key(img),
 jump(img),
 left(img),
 not_have_key(img),
 on_left(obj1,obj2),
 on_left(obj1,obj3),
 on_left(obj1,obj4),
 on_left(obj2,obj1),
 on_left(obj2,obj3),
 on_left(obj2,obj4),
 on_left(obj3,obj1),
 on_left(obj3,obj2),
 on_left(obj3,obj4),
 on_left(obj4,obj1),
 on_left(obj4,obj2),
 on_left(obj4,obj3),
 on_right(obj1,obj2),
 on_right(obj1,obj3),
 on_right(obj1,obj4),
 on_right(obj2,obj1),
 on_right(obj2,obj3),
 on_right(obj2,obj4),
 on_right(obj3,obj1),
 on_right(obj3,obj2),
 on_right(obj3,obj4),
 on_right(obj4,obj1),
 on_right(obj4,obj2),
 on_right(obj4,obj3),
 right(img),
 type(obj1,agent),
 type(obj1,door),
 type(obj1,enemy),
 type(obj1,key),
 type(obj2,agent),
 type(obj2,door),
 type(obj2,enemy),
 type(ob

In [4]:

from percept import SlotAttentionPerceptionModule, YOLOPerceptionModule
from facts_converter import FactsConverter
from nsfr import NSFReasoner
from logic_utils import build_infer_module, generate_atoms
from valuation import RLValuationModule
device = torch.device('cpu')

In [5]:
def get_nsfr_model(lang, clauses, atoms, bk, device):
    PM = YOLOPerceptionModule(e=4, d=11, device=device)
    VM = RLValuationModule(
            lang=lang, device=device)
    FC = FactsConverter(lang=lang, perception_module=PM,
                        valuation_module=VM, device=device)
    IM = build_infer_module(clauses, atoms, lang,
                            m=len(clauses), infer_step=16, device=device)
    # Neuro-Symbolic Forward Reasoner
    NSFR = NSFReasoner(perception_module=PM, facts_converter=FC,
                       infer_module=IM, atoms=atoms, bk=bk, clauses=clauses)
    return NSFR

In [6]:
NSFR = get_nsfr_model(lang, clauses, atoms, bk, device=device)

In [7]:
metric =[[1,0,0,0,11,2],[0,1,0,0,14,2],[0,0,1,0,14.1,2],[0,0,0,1,20,2]]
x = torch.tensor(np.array(metric), dtype=torch.float32).unsqueeze(0)

In [8]:
x.shape

torch.Size([1, 4, 6])

In [9]:
V_T = NSFR(x)

In [10]:
NSFR.print_valuation_batch(V_T)

====== LEARNED PROGRAM ======
C_0:  jump(X):-type(O1,agent),type(O2,enemy),closeby(O1,O2). 1.0
C_1:  left(X):-type(O1,agent),type(O2,key),on_left(O2,O1),not_have_key(X). 1.0
C_2:  right(X):-type(O1,agent),type(O2,key),on_right(O2,O1),not_have_key(X). 1.0
C_3:  left(X):-type(O1,agent),type(O2,door),on_left(O2,O1),have_key(X). 1.0
C_4:  right(X):-type(O1,agent),type(O2,door),on_right(O2,O1),have_key(X). 1.0
===== BATCH:  0 =====
1 .(__T__) :  1.0
57 type(obj4,enemy) :  1.0
52 type(obj3,door) :  1.0
50 type(obj2,key) :  1.0
43 type(obj1,agent) :  1.0
17 not_have_key(img) :  0.9
19 on_left(obj1,obj3) :  0.9
20 on_left(obj1,obj4) :  0.9
22 on_left(obj2,obj3) :  0.9
23 on_left(obj2,obj4) :  0.9
26 on_left(obj3,obj4) :  0.9
41 on_right(obj4,obj3) :  0.9
40 on_right(obj4,obj2) :  0.9
39 on_right(obj4,obj1) :  0.9
33 on_right(obj2,obj1) :  0.9
18 on_left(obj1,obj2) :  0.9
37 on_right(obj3,obj2) :  0.9
36 on_right(obj3,obj1) :  0.9
6 closeby(obj2,obj3) :  0.9
9 closeby(obj3,obj2) :  0.9
42 right

In [11]:
V_T

tensor([[0.06869, 1.00000, 0.06869, 0.06869, 0.06869, 0.06869, 0.90000, 0.06869, 0.06869, 0.90000, 0.06869, 0.06869, 0.06869, 0.06869, 0.06869, 0.09013, 0.09122, 0.90000, 0.90000, 0.90000, 0.90000, 0.06869, 0.90000, 0.90000, 0.06869, 0.06869, 0.90000, 0.06869, 0.06869, 0.06869, 0.06869, 0.06869, 0.06869, 0.90000,
         0.06869, 0.06869, 0.90000, 0.90000, 0.06869, 0.90000, 0.90000, 0.90000, 0.83773, 1.00000, 0.06869, 0.06869, 0.06869, 0.06869, 0.06869, 0.06869, 1.00000, 0.06869, 1.00000, 0.06869, 0.06869, 0.06869, 0.06869, 1.00000, 0.06869]])

In [12]:
atoms

[.(__F__),
 .(__T__),
 closeby(obj1,obj2),
 closeby(obj1,obj3),
 closeby(obj1,obj4),
 closeby(obj2,obj1),
 closeby(obj2,obj3),
 closeby(obj2,obj4),
 closeby(obj3,obj1),
 closeby(obj3,obj2),
 closeby(obj3,obj4),
 closeby(obj4,obj1),
 closeby(obj4,obj2),
 closeby(obj4,obj3),
 have_key(img),
 jump(img),
 left(img),
 not_have_key(img),
 on_left(obj1,obj2),
 on_left(obj1,obj3),
 on_left(obj1,obj4),
 on_left(obj2,obj1),
 on_left(obj2,obj3),
 on_left(obj2,obj4),
 on_left(obj3,obj1),
 on_left(obj3,obj2),
 on_left(obj3,obj4),
 on_left(obj4,obj1),
 on_left(obj4,obj2),
 on_left(obj4,obj3),
 on_right(obj1,obj2),
 on_right(obj1,obj3),
 on_right(obj1,obj4),
 on_right(obj2,obj1),
 on_right(obj2,obj3),
 on_right(obj2,obj4),
 on_right(obj3,obj1),
 on_right(obj3,obj2),
 on_right(obj3,obj4),
 on_right(obj4,obj1),
 on_right(obj4,obj2),
 on_right(obj4,obj3),
 right(img),
 type(obj1,agent),
 type(obj1,door),
 type(obj1,enemy),
 type(obj1,key),
 type(obj2,agent),
 type(obj2,door),
 type(obj2,enemy),
 type(ob

In [13]:
for i, atom in enumerate(atoms):
    print(V_T[0][i], atom)

tensor(0.06869) .(__F__)
tensor(1.) .(__T__)
tensor(0.06869) closeby(obj1,obj2)
tensor(0.06869) closeby(obj1,obj3)
tensor(0.06869) closeby(obj1,obj4)
tensor(0.06869) closeby(obj2,obj1)
tensor(0.90000) closeby(obj2,obj3)
tensor(0.06869) closeby(obj2,obj4)
tensor(0.06869) closeby(obj3,obj1)
tensor(0.90000) closeby(obj3,obj2)
tensor(0.06869) closeby(obj3,obj4)
tensor(0.06869) closeby(obj4,obj1)
tensor(0.06869) closeby(obj4,obj2)
tensor(0.06869) closeby(obj4,obj3)
tensor(0.06869) have_key(img)
tensor(0.09013) jump(img)
tensor(0.09122) left(img)
tensor(0.90000) not_have_key(img)
tensor(0.90000) on_left(obj1,obj2)
tensor(0.90000) on_left(obj1,obj3)
tensor(0.90000) on_left(obj1,obj4)
tensor(0.06869) on_left(obj2,obj1)
tensor(0.90000) on_left(obj2,obj3)
tensor(0.90000) on_left(obj2,obj4)
tensor(0.06869) on_left(obj3,obj1)
tensor(0.06869) on_left(obj3,obj2)
tensor(0.90000) on_left(obj3,obj4)
tensor(0.06869) on_left(obj4,obj1)
tensor(0.06869) on_left(obj4,obj2)
tensor(0.06869) on_left(obj4,obj3)

In [1]:
import argparse

In [8]:
parser = argparse.ArgumentParser()

parser.add_argument("-alg", "--algo", help="Activation function to use",
                    action="store", dest="algo", required=True,
                    choices=['PPO', 'SimplePPO', 'BeamRules'])
parser.add_argument("-env", "--environment", help="Environment to train on",
                    required=True, action="store", dest="env",
                    choices=['coinjump', 'bigfish', 'heist'])
parser.add_argument("-s", "--seed", help="Seed for pytorch + env",
                    required=True, action="store", dest="seed", type=int,
                    default=0)
# parser.add_argument("--recover", help="Recover from the last trained agent",
#                     action="store_true", dest="recover", default=False)
# parser.add_argument("--load", help="Pytorch file to load if continue training",
#                     action="store_true", dest="load", default=False)
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] -alg {PPO,SimplePPO,BeamRules} -env
                             {coinjump,bigfish,heist} -s SEED
ipykernel_launcher.py: error: the following arguments are required: -alg/--algo, -env/--environment, -s/--seed


SystemExit: 2

C:\Users\tongz\anaconda3\envs\procgen\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
print(args.env)

NameError: name 'args' is not defined